In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import annotations

from tqdm.notebook import tqdm
from collections import OrderedDict
import glob
import cv2
import random

from torchvision.io import read_image
import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# Fix random seed

In [ ]:
def fix_seed(seed):
    # random
    random.seed(seed)
#     # Numpy
#     np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
#     # Tensorflow
#     tf.random.set_seed(seed)

SEED = 3407
fix_seed(SEED)

# Load test dataset

In [ ]:
# Test data
test_target = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/test.csv')
test_target["diagnosis"] = "0"
test_target = test_target.astype({"diagnosis": "int64"})

# Define dataset and dataloader

In [ ]:
# # Preprocess images
resize_h = 224
resize_w = 224
input_shape = (resize_h, resize_w)
print(resize_h, resize_w)

In [ ]:
# Define dataset

class CustomImageDataset(Dataset):
    def __init__(self, img_labels: pd.DataFrame, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.img_labels = img_labels

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0] + ".png")
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# mean = imgs.view(3, -1).mean(dim=1)
mean = (0.4138, 0.2210, 0.0737)
print(mean)

In [ ]:
# std = imgs.view(3, -1).std(dim=1)
std = (0.2745, 0.1499, 0.0808)
print(std)

In [ ]:
input_shape = (resize_h, resize_w)
        
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Resize(input_shape),
    transforms.Normalize(mean, std)
])

test_data = CustomImageDataset(test_target, "/kaggle/input/aptos2019-blindness-detection/test_images", data_transforms)

In [ ]:
dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=1,
    shuffle=False
)

# Load model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = torch.load('/kaggle/input/aptos-resnet34-without-prior-fine-tuning/resnet34.pth') # load the entire model with weight parameters, trained with trainval data in addition to fine tuning, so far resnet34 trained with trainval data without fine tuning works best
model = model.to(device)

# Test the model

In [ ]:
model.eval()

t_pred = []
for x, _ in tqdm(dataloader_test):
    
    x = x.to(device)
    y = model.forward(x)
    pred = y.argmax(1)
    preds = pred.to("cpu").detach().numpy()
    t_pred.append(preds)
#     for result in preds:
#         t_pred.append(result)
    
#     acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
#     test_num += acc.size()[0]
#     test_true_num += acc.sum().item()

# print("Accuracy on test set: {:.3f}".format(test_true_num/test_num))

# Submission

In [ ]:
submission = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/sample_submission.csv')
submission['diagnosis'] = np.array(t_pred, dtype='int64').flatten()

submission.head()

In [ ]:
submission['diagnosis'].value_counts()

In [ ]:
submission['diagnosis'].value_counts().plot(kind='bar')
plt.title('Class counts')

In [ ]:
submission.to_csv('submission.csv', index=False)